In [2]:
import pygame
import random

# 색상 정의
COLORS = [
    (0, 0, 0),        # 검정
    (120, 37, 179),   # 보라
    (100, 179, 179),  # 청록
    (80, 34, 22),     # 갈색
    (80, 134, 22),    # 초록
    (180, 34, 22),    # 빨강
    (180, 34, 122),   # 분홍
]

# 테트리스 블록 모양 정의
SHAPES = [
    [[1, 5, 9, 13], [4, 5, 6, 7]],  # I
    [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],  # J
    [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],  # L
    [[1, 2, 5, 6]],  # O
    [[6, 7, 9, 10], [1, 5, 6, 10]],  # S
    [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],  # T
    [[4, 5, 9, 10], [2, 6, 5, 9]]  # Z
]

class Figure:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, len(SHAPES) - 1)
        self.color = random.randint(1, len(COLORS) - 1)
        self.rotation = 0

    def image(self):
        return SHAPES[self.type][self.rotation]

    def rotate(self):
        self.rotation = (self.rotation + 1) % len(SHAPES[self.type])

class Tetris:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.field = []
        self.score = 0
        self.state = "start"
        self.figure = None
        self.next_figure = None
        self.level = 1
        self.x = 100  # 게임 필드 x 좌표
        self.y = 60   # 게임 필드 y 좌표
        self.zoom = 20  # 블록 크기
        
        self.init_game()

    def init_game(self):
        self.field = [[0 for _ in range(self.width)] for _ in range(self.height)]
        self.score = 0
        self.state = "start"
        self.figure = None
        self.spawn_figure()
        self.spawn_next_figure()

    def spawn_figure(self):
        self.figure = self.next_figure if self.next_figure else Figure(3, 0)
        self.spawn_next_figure()

    def spawn_next_figure(self):
        self.next_figure = Figure(3, 0)

    def intersects(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    if (i + self.figure.y > self.height - 1 or
                        j + self.figure.x > self.width - 1 or
                        j + self.figure.x < 0 or
                        self.field[i + self.figure.y][j + self.figure.x] > 0):
                        return True
        return False

    def break_lines(self):
        lines = 0
        for i in range(1, self.height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
            if zeros == 0:
                lines += 1
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.field[i1][j] = self.field[i1-1][j]
        self.score += lines ** 2
        if self.score > self.level * 10:
            self.level += 1

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.break_lines()
        self.spawn_figure()
        if self.intersects():
            self.state = "gameover"

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        if self.intersects():
            self.figure.x = old_x

    def rotate(self):
        old_rotation = self.figure.rotation
        self.figure.rotate()
        if self.intersects():
            self.figure.rotation = old_rotation
    def go_space(self):
        # 블록을 바닥까지 즉시 이동
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

def main():
    # 초기화
    pygame.init()
    pygame.display.set_caption("Tetris")
    
    # 화면 설정
    size = (400, 500)
    screen = pygame.display.set_mode(size)
    
    # 폰트 설정
    font = pygame.font.SysFont('Calibri', 25, True, False)
    font_small = pygame.font.SysFont('Calibri', 18, True, False)

    # 게임 변수 초기화
    clock = pygame.time.Clock()
    fps = 25
    game = Tetris(20, 10)
    counter = 0
    pressing_down = False

    while True:
        if game.figure is None:
            game.spawn_figure()
        counter += 1
        if counter > 100000:
            counter = 0

        if counter % (fps // game.level // 2) == 0 or pressing_down:
            if game.state == "start":
                game.go_down()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    game.rotate()
                if event.key == pygame.K_DOWN:
                    pressing_down = True
                if event.key == pygame.K_LEFT:
                    game.go_side(-1)
                if event.key == pygame.K_RIGHT:
                    game.go_side(1) 
                if event.key == pygame.K_SPACE:
                    game.go_space()
                if event.key == pygame.K_ESCAPE:
                    game.__init__(20, 10)

            if event.type == pygame.KEYUP:
                if event.key == pygame.K_DOWN:
                    pressing_down = False

        # 화면 그리기
        screen.fill(COLORS[0])

        # 게임 필드 그리기
        for i in range(game.height):
            for j in range(game.width):
                pygame.draw.rect(screen, COLORS[game.field[i][j]],
                               [game.x + game.zoom * j, game.y + game.zoom * i, 
                                game.zoom-1, game.zoom-1])

        # 현재 블록 그리기
        if game.figure is not None:
            for i in range(4):
                for j in range(4):
                    p = i * 4 + j
                    if p in game.figure.image():
                        pygame.draw.rect(screen, COLORS[game.figure.color],
                                       [game.x + game.zoom * (j + game.figure.x),
                                        game.y + game.zoom * (i + game.figure.y),
                                        game.zoom-1, game.zoom-1])

        # 점수와 레벨 표시
        text_score = font.render("Score: " + str(game.score), True, (255, 255, 255))
        text_level = font.render("Level: " + str(game.level), True, (255, 255, 255))
        screen.blit(text_score, [0, 0])
        screen.blit(text_level, [0, 25])

        # 게임오버 표시
        if game.state == "gameover":
            game_over_text = font.render("Game Over! Press ESC", True, (255, 255, 255))
            screen.blit(game_over_text, [20, 200])

        pygame.display.flip()
        clock.tick(fps)

if __name__ == '__main__':
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
